In [16]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

def get_beta_value(stock_code, debug=False):
    """
    네이버 금융에서 특정 종목의 52주 베타값을 크롤링
    
    Args:
        stock_code (str): 종목코드 (예: '096770')
        debug (bool): 디버그 모드
    
    Returns:
        float or None: 52주 베타값, 실패시 None
    """
    # 투자정보 페이지 URL (베타값이 있는 곳)
    url = f"https://finance.naver.com/item/invest.naver?code={stock_code}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate',
        'Referer': 'https://finance.naver.com/',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if debug:
            print(f"페이지 제목: {soup.title.get_text() if soup.title else 'None'}")
        
        # 방법 1: 모든 테이블에서 베타 찾기 (더 정확한 검색)
        tables = soup.find_all('table')
        for table_idx, table in enumerate(tables):
            if debug:
                print(f"테이블 {table_idx} 검사 중...")
            
            rows = table.find_all('tr')
            for row_idx, row in enumerate(rows):
                row_text = row.get_text(strip=True)
                if debug and ('베타' in row_text or 'Beta' in row_text):
                    print(f"  행 {row_idx}에서 베타 관련 텍스트 발견: {row_text[:100]}...")
                
                cells = row.find_all(['th', 'td'])
                for i, cell in enumerate(cells):
                    cell_text = cell.get_text(strip=True)
                    
                    # 베타 관련 키워드가 포함된 셀 찾기
                    if any(keyword in cell_text.replace(' ', '') for keyword in ['52주베타', '베타(52주)', 'Beta', 'beta']):
                        if debug:
                            print(f"    베타 관련 셀 발견: '{cell_text}'")
                        
                        # 현재 셀에서 직접 숫자 찾기
                        number_match = re.search(r'(\d+\.\d+)', cell_text)
                        if number_match:
                            try:
                                beta_value = float(number_match.group(1))
                                if debug:
                                    print(f"    현재 셀에서 베타값 발견: {beta_value}")
                                return beta_value
                            except ValueError:
                                pass
                        
                        # 같은 행의 다음 셀들에서 숫자 찾기
                        for j in range(i + 1, len(cells)):
                            next_cell_text = cells[j].get_text(strip=True)
                            if debug:
                                print(f"    다음 셀 확인: '{next_cell_text}'")
                            
                            # 숫자만 있는 셀 찾기
                            if re.match(r'^\d+\.\d+
        
        # 방법 2: 정규표현식으로 전체 페이지에서 베타 관련 패턴 찾기
        page_text = soup.get_text()
        
        # 52주 베타 패턴들
        beta_patterns = [
            r'52주\s*베타[:\s]*(\d+\.\d+)',
            r'52주베타[:\s]*(\d+\.\d+)',
            r'Beta[:\s]*(\d+\.\d+)',
            r'beta[:\s]*(\d+\.\d+)',
            r'베타[:\s]*(\d+\.\d+)'
        ]
        
        for pattern in beta_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                try:
                    beta_value = float(matches[0])
                    if debug:
                        print(f"정규표현식으로 베타값 발견: {beta_value}")
                    return beta_value
                except ValueError:
                    continue
        
        # 방법 3: 특정 클래스나 ID를 가진 요소에서 찾기
        possible_selectors = [
            'td.num',
            'span.num',
            '.tb_type1 td',
            '.tb_type1_ifrs td',
            '.section_trade_compare td'
        ]
        
        for selector in possible_selectors:
            elements = soup.select(selector)
            for elem in elements:
                elem_text = elem.get_text(strip=True)
                # 베타값 범위 (일반적으로 0~5 사이)
                if re.match(r'^\d+\.\d+$', elem_text):
                    try:
                        value = float(elem_text)
                        if 0 <= value <= 5:  # 베타값 범위
                            # 주변 텍스트에서 베타 관련 키워드 확인
                            parent_text = elem.parent.get_text() if elem.parent else ""
                            if any(keyword in parent_text for keyword in ['베타', 'Beta', 'beta']):
                                if debug:
                                    print(f"베타값으로 추정되는 값 발견: {value}")
                                return value
                    except ValueError:
                        continue
        
        if debug:
            print("모든 방법으로 베타값을 찾을 수 없습니다.")
            # 페이지 내용의 일부 출력 (디버깅용)
            print("페이지 내용 샘플:")
            print(page_text[:1000])
        
        print(f"종목코드 {stock_code}: 52주 베타값을 찾을 수 없습니다.")
        return None
        
    except requests.RequestException as e:
        print(f"종목코드 {stock_code}: 요청 중 오류 발생 - {e}")
        return None
    except Exception as e:
        print(f"종목코드 {stock_code}: 파싱 중 오류 발생 - {e}")
        return None

def get_multiple_beta_values(stock_codes, delay=1):
    """
    여러 종목의 52주 베타값을 크롤링
    
    Args:
        stock_codes (list): 종목코드 리스트
        delay (float): 요청 간 지연시간 (초)
    
    Returns:
        dict: {종목코드: 베타값} 형태의 딕셔너리
    """
    results = {}
    
    for i, code in enumerate(stock_codes):
        print(f"진행상황: {i+1}/{len(stock_codes)} - 종목코드: {code}")
        
        beta = get_beta_value(code, debug=False)
        results[code] = beta
        
        if beta is not None:
            print(f"  -> 52주 베타: {beta}")
        
        # 마지막 요청이 아닌 경우 지연
        if i < len(stock_codes) - 1:
            time.sleep(delay)
    
    return results

def save_to_csv(results, filename="beta_values.csv"):
    """
    결과를 CSV 파일로 저장
    
    Args:
        results (dict): {종목코드: 베타값} 딕셔너리
        filename (str): 저장할 파일명
    """
    df = pd.DataFrame(list(results.items()), columns=['종목코드', '52주베타'])
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"결과가 {filename}에 저장되었습니다.")

# 사용 예시
if __name__ == "__main__":
    # 단일 종목 조회 (디버그 모드로)
    print("=== 단일 종목 조회 (디버그 모드) ===")
    beta = get_beta_value("096770", debug=True)
    print(f"096770의 52주 베타: {beta}")
    
    # 여러 종목 조회
    print("\n=== 여러 종목 조회 ===")
    stock_codes = ["096770", "005930", "000660", "035420", "051910"]  # 예시 종목코드들
    
    results = get_multiple_beta_values(stock_codes, delay=1)
    
    # 결과 출력
    print("\n=== 최종 결과 ===")
    for code, beta in results.items():
        print(f"{code}: {beta}")
    
    # CSV 파일로 저장
    save_to_csv(results), next_cell_text):
                                try:
                                    beta_value = float(next_cell_text)
                                    if 0 <= beta_value <= 10:  # 베타값 범위 확장
                                        if debug:
                                            print(f"    베타값 발견: {beta_value}")
                                        return beta_value
                                except ValueError:
                                    continue
                            
                            # 숫자 패턴 찾기
                            number_match = re.search(r'(\d+\.\d+)', next_cell_text)
                            if number_match:
                                try:
                                    beta_value = float(number_match.group(1))
                                    if 0 <= beta_value <= 10:
                                        if debug:
                                            print(f"    베타값 발견: {beta_value}")
                                        return beta_value
                                except ValueError:
                                    continue
        
        # 방법 2: 정규표현식으로 전체 페이지에서 베타 관련 패턴 찾기
        page_text = soup.get_text()
        
        # 52주 베타 패턴들
        beta_patterns = [
            r'52주\s*베타[:\s]*(\d+\.\d+)',
            r'52주베타[:\s]*(\d+\.\d+)',
            r'Beta[:\s]*(\d+\.\d+)',
            r'beta[:\s]*(\d+\.\d+)',
            r'베타[:\s]*(\d+\.\d+)'
        ]
        
        for pattern in beta_patterns:
            matches = re.findall(pattern, page_text, re.IGNORECASE)
            if matches:
                try:
                    beta_value = float(matches[0])
                    if debug:
                        print(f"정규표현식으로 베타값 발견: {beta_value}")
                    return beta_value
                except ValueError:
                    continue
        
        # 방법 3: 특정 클래스나 ID를 가진 요소에서 찾기
        possible_selectors = [
            'td.num',
            'span.num',
            '.tb_type1 td',
            '.tb_type1_ifrs td',
            '.section_trade_compare td'
        ]
        
        for selector in possible_selectors:
            elements = soup.select(selector)
            for elem in elements:
                elem_text = elem.get_text(strip=True)
                # 베타값 범위 (일반적으로 0~5 사이)
                if re.match(r'^\d+\.\d+$', elem_text):
                    try:
                        value = float(elem_text)
                        if 0 <= value <= 5:  # 베타값 범위
                            # 주변 텍스트에서 베타 관련 키워드 확인
                            parent_text = elem.parent.get_text() if elem.parent else ""
                            if any(keyword in parent_text for keyword in ['베타', 'Beta', 'beta']):
                                if debug:
                                    print(f"베타값으로 추정되는 값 발견: {value}")
                                return value
                    except ValueError:
                        continue
        
        if debug:
            print("모든 방법으로 베타값을 찾을 수 없습니다.")
            # 페이지 내용의 일부 출력 (디버깅용)
            print("페이지 내용 샘플:")
            print(page_text[:1000])
        
        print(f"종목코드 {stock_code}: 52주 베타값을 찾을 수 없습니다.")
        return None
        
    except requests.RequestException as e:
        print(f"종목코드 {stock_code}: 요청 중 오류 발생 - {e}")
        return None
    except Exception as e:
        print(f"종목코드 {stock_code}: 파싱 중 오류 발생 - {e}")
        return None

def get_multiple_beta_values(stock_codes, delay=1):
    """
    여러 종목의 52주 베타값을 크롤링
    
    Args:
        stock_codes (list): 종목코드 리스트
        delay (float): 요청 간 지연시간 (초)
    
    Returns:
        dict: {종목코드: 베타값} 형태의 딕셔너리
    """
    results = {}
    
    for i, code in enumerate(stock_codes):
        print(f"진행상황: {i+1}/{len(stock_codes)} - 종목코드: {code}")
        
        beta = get_beta_value(code, debug=False)
        results[code] = beta
        
        if beta is not None:
            print(f"  -> 52주 베타: {beta}")
        
        # 마지막 요청이 아닌 경우 지연
        if i < len(stock_codes) - 1:
            time.sleep(delay)
    
    return results

def save_to_csv(results, filename="beta_values.csv"):
    """
    결과를 CSV 파일로 저장
    
    Args:
        results (dict): {종목코드: 베타값} 딕셔너리
        filename (str): 저장할 파일명
    """
    df = pd.DataFrame(list(results.items()), columns=['종목코드', '52주베타'])
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"결과가 {filename}에 저장되었습니다.")

# 사용 예시
if __name__ == "__main__":
    # 단일 종목 조회 (디버그 모드로)
    print("=== 단일 종목 조회 (디버그 모드) ===")
    beta = get_beta_value("096770", debug=True)
    print(f"096770의 52주 베타: {beta}")
    
    # 여러 종목 조회
    print("\n=== 여러 종목 조회 ===")
    stock_codes = ["096770", "005930", "000660", "035420", "051910"]  # 예시 종목코드들
    
    results = get_multiple_beta_values(stock_codes, delay=1)
    
    # 결과 출력
    print("\n=== 최종 결과 ===")
    for code, beta in results.items():
        print(f"{code}: {beta}")
    
    # CSV 파일로 저장
    save_to_csv(results)

=== 단일 종목 조회 (디버그 모드) ===
페이지 제목: SK이노베이션 : 네이버페이 증권
모든 방법으로 베타값을 찾을 수 없습니다.
페이지 내용 샘플:



SK이노베이션 : 네이버페이 증권


































메인 메뉴로 바로가기
본문으로 바로가기










네이버





페이






증권






증권 종목명·지수명 검색







검색


자동완성







@code@
@txt@
@market@

@full_txt@
@in_code@
@in_name@
@in_link@
@in_market@




												공모주와 해외 종목은 모바일 페이지로 이동합니다.
											








												현재 자동완성 기능을 사용하고 계십니다.
											









												자동완성 기능이 활성화되었습니다.
											





















증권 홈
국내증시선택됨
해외증시
시장지표
리서치
뉴스
MY























종목 시세 정보
2025년 06월
        12일 16시
        10분 기준 장마감
종목명 SK이노베이션
종목코드 096770 코스피
현재가 95,500 전일대비 상승 1,300
            플러스
            1.38 퍼센트
        
전일가 94,200
시가 94,700
고가 97,500
상한가 122,400
저가 94,300
하한가 66,000
거래량 547,944
거래대금 52,448백만



SK이노베이션


096770

날짜

2025.06.12 기준(KRX 장마감)


실시간



기업개요



기업개요
동사는 2007년 SK이노베이션으로 설립되어 석유·화학, LNG, 전력, 배터리를 아우르는 에너지 포트폴리오를 구축하였음.
2024년 SK E&S와 합병으로 아시아·태평양 민간 최대 종합 에너지 회사로 거듭났으며, 석유 55%, E&

In [ ]:
import os
import requests
from dotenv import load_dotenv

# .env에서 API 키 로드
load_dotenv()
api_key = os.getenv("KRX_API_KEY")

url = "https://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo"
params = {
    "serviceKey": api_key,
    "numOfRows": 5,        # 가져올 행 수 (예: 5)
    "pageNo": 1,
    "resultType": "json",
    "basDt": "20240617",   # 기준 일자
    "itmsNm": "삼성전자",  # 종목명
}

response = requests.get(url, params=params)
data = response.json()
print(data["response"]["body"]["items"]["item"][0].keys())
